### Extract all data into dataframe in jupyter notebook: 

In [1]:
# Modules: 
import pandas as pd 
import numpy as np 
from datetime import datetime
from sqlalchemy import create_engine 
from config import user, password
import math
from datetime import datetime, timedelta

In [2]:
user

'postgres'

In [3]:
# File path: 
google_file = "Resources/GOOG.csv"
acquisitions_file = "Resources/acquisitions.csv"

# Create dataframe: 
google_df = pd.read_csv(google_file)
acquisitions_df = pd.read_csv(acquisitions_file, dtype = {'AcquisitionMonthDate': str, "AcquisitionYear": str})
acquisitions_df.dropna(subset = ['AcquisitionMonthDate', 'AcquisitionMonth', 'AcquisitionYear'], inplace=True)
google_df = google_df.set_index('Date')
google_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2004-08-19,49.813286,51.835709,47.800831,49.982655,49.982655,44871300
2004-08-20,50.316402,54.336334,50.062355,53.952770,53.952770,22942800
2004-08-23,55.168217,56.528118,54.321388,54.495735,54.495735,18342800
2004-08-24,55.412300,55.591629,51.591621,52.239193,52.239193,15319700
2004-08-25,52.284027,53.798351,51.746044,52.802086,52.802086,9232100


In [4]:
acquisitions_df.head()



,AcquisitionID,AcquisitionMonth,AcquisitionMonthDate,AcquisitionYear,Company,Business,Country,Value (USD),Derived products,ParentCompany
0,ACQ99,November,11,2015,bebop,Cloud software,USA,380000000.0,Google Cloud Platform,Google
1,ACQ98,November,11,2015,Fly Labs,Video editing,USA,NaN,Google Photos,Google
2,ACQ97,December,8,2015,Clearleap,Cloud-based video management,USA,NaN,NaN,IBM
3,ACQ96,December,18,2015,Metanautix,Big Data Analytics,USA,NaN,NaN,Microsoft
4,ACQ95,December,21,2015,"Talko, Inc.",Mobile communications,USA,NaN,NaN,Microsoft


### Data cleaning for acquisitions dataframe: 

In [5]:

# Extract only Google Acquisitions: 
google_acquisitions_df = acquisitions_df.loc[acquisitions_df["ParentCompany"] == "Google", :]
google_acquisitions_df.head()

google_acquisitions_df = google_acquisitions_df.sort_values("AcquisitionYear")


# Drop all the NaN value: 
google_acquisitions_df.dropna(how='any')
google_acquisitions_df['AcquisitionMonthDate'][116]

# Reset index for Google Acquisitions: 
google_acquisitions_df = google_acquisitions_df.reset_index()
google_acquisitions_df.head()


,index,AcquisitionID,AcquisitionMonth,AcquisitionMonthDate,AcquisitionYear,Company,Business,Country,Value (USD),Derived products,ParentCompany
0,101,ACQ831,February,12,2001,Deja,Usenet,USA,NaN,Google Groups,Google
1,116,ACQ818,September,20,2001,Outride,Web search engine,USA,NaN,Google Personalized Search,Google
2,160,ACQ779,September,30,2003,Kaltix,Web search engine,USA,NaN,iGoogle,Google
3,174,ACQ766,May,10,2004,Ignite Logic,HTML editor,USA,NaN,Google Sites,Google
4,177,ACQ763,July,13,2004,Picasa,Image organizer,USA,NaN,"Picasa, Blogger",Google


In [6]:
google_acquisitions_df.columns

Index(['index', 'AcquisitionID', 'AcquisitionMonth', 'AcquisitionMonthDate',
       'AcquisitionYear', 'Company', 'Business', 'Country', 'Value (USD)',
       'Derived products', 'ParentCompany'],
      dtype='object')

In [7]:
# Transform all date , month, year data to one column in datetime form: 

# Create a list that store Date, Month, Year Value: 
date_str_list = []
for x in range(0, len(google_acquisitions_df.index)):
    date_str = google_acquisitions_df['AcquisitionMonthDate'][x] + " " + google_acquisitions_df['AcquisitionMonth'][x] + ", " + google_acquisitions_df['AcquisitionYear'][x]
    d = datetime.strptime(date_str, '%d %B, %Y')
    d_str = d.strftime('%Y-%m-%d')
    date_str_list.append(d_str)

# Add that list as a new column:""
google_acquisitions_df["Date"] = date_str_list
# Reset index into Date: 
google_acquisitions_df = google_acquisitions_df.set_index('Date')

# Delete columns to clean data: 
google_acquisitions_df = google_acquisitions_df.drop(columns=['index', 'Value (USD)', 'AcquisitionID', 'AcquisitionMonth', 'AcquisitionMonthDate', 'AcquisitionYear'])


In [8]:
google_acquisitions_df.head()

,Company,Business,Country,Derived products,ParentCompany
Date,,,,,
2001-02-12,Deja,Usenet,USA,Google Groups,Google
2001-09-20,Outride,Web search engine,USA,Google Personalized Search,Google
2003-09-30,Kaltix,Web search engine,USA,iGoogle,Google
2004-05-10,Ignite Logic,HTML editor,USA,Google Sites,Google
2004-07-13,Picasa,Image organizer,USA,"Picasa, Blogger",Google


In [9]:
google_acquisitions_df.count()

Company             204
Business            204
Country             201
Derived products    202
ParentCompany       204
dtype: int64

### Data cleaning for Google Stock Price Dataframe: 

In [10]:
# Extract only neccessary data like Close Price, Volume: 
google_df = google_df.drop(columns=['Open','High', 'Low','Close'])
google_df.head()


,Adj Close,Volume
Date,,
2004-08-19,49.982655,44871300
2004-08-20,53.952770,22942800
2004-08-23,54.495735,18342800
2004-08-24,52.239193,15319700
2004-08-25,52.802086,9232100


In [11]:
google_df.count()

Adj Close    3838
Volume       3838
dtype: int64

### Loading Data into sql Database: 

In [12]:
# Connect to local database
connection_string = user + ":" + password + "@localhost:5432/acquisition_db"
engine = create_engine(f'postgresql://{connection_string}')


In [13]:
# Check for tables
engine.table_names()

['acquisition_target', 'google_stock_price']

In [14]:
google_acquisitions_df.to_sql(name='acquisition_target', con=engine, if_exists='append', index=True)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "Date" of relation "acquisition_target" does not exist
LINE 1: INSERT INTO acquisition_target ("Date", "Company", "Business...
                                        ^

[SQL: INSERT INTO acquisition_target ("Date", "Company", "Business", "Country", "Derived products", "ParentCompany") VALUES (%(Date)s, %(Company)s, %(Business)s, %(Country)s, %(Derived products)s, %(ParentCompany)s)]
[parameters: ({'Date': '2001-02-12', 'Company': 'Deja', 'Business': 'Usenet', 'Country': 'USA', 'Derived products': 'Google Groups', 'ParentCompany': 'Google'}, {'Date': '2001-09-20', 'Company': 'Outride', 'Business': 'Web search engine', 'Country': 'USA', 'Derived products': 'Google Personalized Search', 'ParentCompany': 'Google'}, {'Date': '2003-09-30', 'Company': 'Kaltix', 'Business': 'Web search engine', 'Country': 'USA', 'Derived products': 'iGoogle', 'ParentCompany': 'Google'}, {'Date': '2004-05-10', 'Company': 'Ignite Logic', 'Business': 'HT

In [15]:
google_df.to_sql(name='google_stock_price', con=engine, if_exists='append', index=False)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "Adj Close" of relation "google_stock_price" does not exist
LINE 1: INSERT INTO google_stock_price ("Adj Close", "Volume") VALUE...
                                        ^

[SQL: INSERT INTO google_stock_price ("Adj Close", "Volume") VALUES (%(Adj Close)s, %(Volume)s)]
[parameters: ({'Adj Close': 49.982655, 'Volume': 44871300}, {'Adj Close': 53.952769999999994, 'Volume': 22942800}, {'Adj Close': 54.495734999999996, 'Volume': 18342800}, {'Adj Close': 52.239193, 'Volume': 15319700}, {'Adj Close': 52.802085999999996, 'Volume': 9232100}, {'Adj Close': 53.753517, 'Volume': 7128600}, {'Adj Close': 52.87680400000001, 'Volume': 6241200}, {'Adj Close': 50.814533000000004, 'Volume': 5221400}  ... displaying 10 of 3838 total bound parameter sets ...  {'Adj Close': 1298.0, 'Volume': 826700}, {'Adj Close': 1311.459961, 'Volume': 1193500})]
(Background on this error at: http://sqlalche.me/e/f405)

In [16]:
# Confirm data has been added by querying the customer_name table
pd.read_sql_query('select * from acquisition_target', con=engine).head()

,id,acquired_date,acquired_company,acquired_business,country,product


In [17]:
# Confirm data has been added by querying the customer_location table
pd.read_sql_query('select * from google_stock_price', con=engine).head()

,id,stock_date,closing_price,trading_volume
